In [5]:
import time
import tracemalloc
from typing import Dict, List, Tuple
from collections import defaultdict

class Node:
    """
    Lớp đại diện cho một nút trong cây UHUFP (Cây mẫu tần suất tiện ích cao không chắc chắn).

    Thuộc tính:
        item (str hoặc None): Mục được lưu trữ trong nút này. Mặc định là None.
        prob (float): Xác suất liên quan đến nút này. Mặc định là 0.0.
        count (int): Số lần (hỗ trợ) của nút này. Mặc định là 0.
        utility (int): Giá trị tiện ích của mục của nút này. Mặc định là 0.
        nu (int): Tiện ích của nút, là giá trị tiện ích ước tính cho nút này. Mặc định là 0.
        children (dict): Từ điển lưu trữ các nút con, trong đó khóa là mục và giá trị là nút con tương ứng.
        parent (Node hoặc None): Tham chiếu đến nút cha của nút này. Mặc định là None.
    """

    def __init__(self, item=None, prob=0.0, count=0, utility=0, nu=0):
        """
        Khởi tạo một nút mới trong cây UHUFP.
        """
        self.item = item
        self.prob = prob
        self.count = count
        self.utility = utility
        self.children = {}
        self.parent = None
        self.nu = nu

class UHUFP_tree:
    """
    Lớp đại diện cho cây UHUFP (Cây mẫu tần suất tiện ích cao không chắc chắn).

    Thuộc tính:
        root (Node): Gốc của cây, là một đối tượng Node.
        header_table (defaultdict): Bảng tiêu đề, lưu trữ các liên kết đến các nút với cùng một mục.
    """

    def __init__(self):
        """
        Khởi tạo cây UHUFP với gốc là một nút rỗng và bảng tiêu đề là một defaultdict.
        """
        self.root = Node()
        self.header_table = defaultdict(list)

    def insert_reorganized_transaction(self, transaction, utilities, sorted_items):
        """
        Chèn một giao dịch đã được tổ chức lại vào cây UHUFP.

        Tham số:
            transaction (dict): Giao dịch với các mục và xác suất tương ứng.
            utilities (dict): Bảng tiện ích của các mục trong giao dịch.
            sorted_items (list): Danh sách các mục được sắp xếp theo thứ tự TWU giảm dần.
        """
        current_node = self.root
        rtu = sum(utilities.values())  # Tổng tiện ích của giao dịch

        for i, item in enumerate(sorted_items):
            item_prob = transaction[item]
            item_utility = utilities[item]
            
            if (item, item_prob, item_utility) in current_node.children:
                child_node = current_node.children[(item, item_prob, item_utility)]
                child_node.count += 1
                items_after = sorted_items[i+1:]
                remaining_utility = sum(utilities.get(it, 0) for it in items_after)
                child_node.nu += rtu - remaining_utility
            else:
                child_node = Node(item=item, prob=item_prob, count=1, utility=item_utility)
                items_after = sorted_items[i+1:]
                remaining_utility = sum(utilities.get(it, 0) for it in items_after)
                child_node.nu = rtu - remaining_utility
                
                child_node.parent = current_node
                current_node.children[(item, item_prob, item_utility)] = child_node
                self.header_table[item].append(child_node)
            
            current_node = child_node

    def insert_reorganized_path(self, transaction, utilities, sorted_items, miu):
        """
        Chèn một đường dẫn đã được tổ chức lại vào cây UHUFP. (chiến lược 8)

        Tham số:
            transaction (dict): Giao dịch với các mục và xác suất tương ứng.
            utilities (dict): Bảng tiện ích của các mục trong giao dịch.
            sorted_items (list): Danh sách các mục được sắp xếp theo thứ tự tiện ích đường dẫn của item trong CPB giảm dần.
            miu (dict): Bảng tiện ích mục nhỏ nhất cho mỗi mục trong đường dẫn.
        """
        current_node = self.root
        for i, item in enumerate(sorted_items):
            item_prob = transaction[item]
            item_utility = utilities[item]

            if (item, item_prob, item_utility) in current_node.children:
                child_node = current_node.children[(item, item_prob, item_utility)]
                child_node.count += 1
                utility_after = sum(miu.get(x, 0) * child_node.count for x in list(sorted_items.keys())[i + 1:])
                child_node.nu += sorted_items[item] - utility_after
            else:
                child_node = Node(item=item, prob=item_prob, count=1, utility=item_utility)
                utility_after = sum(miu.get(x, 0) * child_node.count for x in list(sorted_items.keys())[i + 1:])
                child_node.nu = sorted_items[item] - utility_after

                child_node.parent = current_node
                current_node.children[(item, item_prob, item_utility)] = child_node
                self.header_table[item].append(child_node)

            current_node = child_node

    def print_header_table(self):
        """
        In bảng tiêu đề (header table) của cây UHUFP.
        """
        print("\nHeader Table:")
        for item, nodes in self.header_table.items():
            for node in nodes:
                print(f"  -> Node: Item={node.item}, Support={node.prob}, Count={node.count}, Utility={node.utility}")
        print("End of Header Table\n")

    def print_tree(self):
        """
        In cấu trúc của cây UHUFP.
        """
        def print_node(node, level=0):
            indent = ' ' * (level * 4)
            print(f"{indent}Item: {node.item}, Support: {node.prob}, Count: {node.count}, Utility: {node.utility}, Nu: {node.nu}")
            for child in node.children.values():
                print_node(child, level + 1)

        print("UHUFP Tree:")
        print_node(self.root)
        print("End of UHUFP Tree")

    def prune_patterns(self, k, item_twu, minUtil):
        """
        Cắt bớt các mẫu không cần thiết dựa trên TWU và số lượng top-k.

        Tham số:
            k (int): Số lượng các mục top-k cần giữ lại.
            item_twu (dict): Bảng TWU của các mục.
            minUtil (int): Ngưỡng tiện ích
        """
        item_exp_sups = {}
        for item, nodes in self.header_table.items():
            exp_sup = sum(node.prob * node.count for node in nodes)
            item_exp_sups[item] = exp_sup
        # Sắp xếp theo expSup của item
        sorted_items_by_exp_sup = sorted(self.header_table.keys(), key=lambda item: item_exp_sups[item], reverse=True)
        self.header_table = {item: self.header_table[item] for item in sorted_items_by_exp_sup}

        filtered_header_table = defaultdict(list)
        for item, nodes in self.header_table.items():
            k -= 1
            twu = item_twu[item]
            if twu >= minUtil:
                filtered_header_table[item] = self.header_table[item]
            if k <= 0:
                break
        self.header_table = filtered_header_table

    def prune_tree(self, node):
        """
        Loại bỏ các nút khỏi cây nếu mục của chúng không có trong bảng tiêu đề.

        Tham số:
            node (Node): Nút hiện tại đang xử lý để cắt tỉa.
        """
        items_to_keep = set(self.header_table.keys())
        children_to_keep = {k: v for k, v in node.children.items() if v.item in items_to_keep}
            
        node.children = children_to_keep
            
        for child in node.children.values():
            self.prune_tree(child)


class TUHUFP_Growth:
    """Lớp này triển khai thuật toán khai thác top-k mẫu phổ biến tiện ích cao không chắc chắn
    """

    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.peak_memory_usage = 0
        self.top_k_UHUFP = []
        self.minUtil = 0

    
    def calculate_expected_support(self, paths, prefix):
        """Tính độ hỗ trợ mong đợi của mẫu (prefix) cụ thể thông qua đường dẫn đã thu thập

        Args:
            paths (Tuple(Tuple(List[str,float,int,int], int))): đường dẫn của prefix
            prefix (List[str]): mẫu cần tính giá trị tiện ích
        Returns:
            int: giá trị độ hỗ trợ mong đợi của prefix
        """

        total_expected_support = 0.0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                path_support = 1.0
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        path_support = round((path_support*i[1]),2) 
                    else:
                        path_support = 0
                        break

                total_expected_support += path_support * count 

        return round(total_expected_support, 2)




    def calculate_utility(self, paths, prefix):
        """Tính giá trí tiện ích của mẫu (prefix) cụ thể thông qua đường dẫn đã thu thập

        Args:
            paths (Tuple(Tuple(List[str,float,int,int], int))): đường dẫn của prefix
            prefix (List[str]): mẫu cần tính giá trị tiện ích
        Returns:
            int: giá trị tiện ích của prefix
        """

        prefix_utility = 0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                sum_utility = 0
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        sum_utility += i[2] 
                    else:
                        sum_utility = 0
                        break

                prefix_utility += sum_utility * count 

        return prefix_utility

        
 
    def calculate_twu(self, paths, prefix):
        """Tính TWU của prefix 

        Args:
            paths (Tuple(Tuple(List[str,float,int,int], int))): đường dẫn của prefix
            prefix (List[str]): mẫu cần tính TWU
        Returns:
            int: giá trị TWU
        """
        prefix_twu = 0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                sum_twu= 0
                # Tính expected support cho path bằng cách nhân các xác suất (prob) của các items trong prefix
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        sum_twu = i[3]  # Nhân xác suất (prob) của item trong path

                prefix_twu += sum_twu * count  # Nhân với count của path

        return prefix_twu


    def mine_top_k_patterns(self, uhufp_tree, k, minSup, minUtil, expected_support, prefix=None, top_k=None):
        """ Phương thức chính khai thác các tập mẫu UHUFP

        Args:
            uhufp_tree (UHUFP_tree): cây UHUFP của mẫu 
            k (int): số lượng mẫu cần tìm
            minSup (float): ngưỡng phổ biến trong top-k
            minUtil (int): ngưỡng tiện ích
            expected_support (float): giá trị độ hỗ trợ mong đợi của prefix, ban đâu là 0
            prefix (List[str]): là tiền tố của mẫu. Mặc đinh ban đầu None.
            top_k (List[UHUFP], ): danh sách chứa k mẫu UHUFP. Mặc đinh ban đầu None.

        Returns:
            List[UHUFP]: danh sách chứa k mẫu UHUFP.
        """
        if top_k is None:
            top_k = []
        if prefix is None:
            prefix = []
        if len(top_k) == k and top_k is not None:
            minSup = top_k[-1][1]

        #Duyệt từng phần từ trong bảng header
        for item, nodes in uhufp_tree.header_table.items():
            # kiểm tra item khác prefix
            if item not in prefix:
                max_prob = max(node.prob for node in nodes if node.item == item)
                overestimate = max_prob*expected_support
                # chiến lược 4
                if overestimate < minSup:
                    return top_k
                # Tạo mẫu mới từ prefix và item
                new_prefix = [item] + prefix

                if any(existing_pattern[0] == tuple(sorted(new_prefix)) for existing_pattern in top_k):
                    continue
                # print(f"ITEM: {new_prefix}")
                # thu thập conditional pattern base của mẫu
                conditional_pattern_base, miu, path_util_item = self.build_conditional_pattern_base(uhufp_tree, new_prefix, minUtil)
                # chiến lược 8
                if path_util_item >= minUtil:
                    # Tính độ hỗ trợ mong đợi
                    expected_support = self.calculate_expected_support(conditional_pattern_base, new_prefix)
            
                    # Tính giá trị tiện ích
                    utility = self.calculate_utility(conditional_pattern_base, new_prefix)
                    # xây dựng cây UHUFP cho new_prefix
                    conditional_pattern_tree = self.build_conditional_tree(conditional_pattern_base, miu)
                    # conditional_pattern_tree.print_tree()
                    # conditional_pattern_tree.print_header_table()
                    
                    # print(f"{new_prefix} : {expected_support} : {utility}")
                    
                    # kiểm tra độ hỗ trợ mong đợi với ngưỡng phổ biến và ngưỡng tiện ích
                    if expected_support > minSup:
                        if utility >= minUtil:
                            # Add the pattern to top_k list if necessary
                            top_k.append((tuple(sorted(new_prefix)), expected_support, utility))
                            # Sort and maintain top-k
                            top_k.sort(key=lambda x: (x[1], x[2]), reverse=True)
                            if len(top_k) > k:
                                top_k.pop()
                            if len(top_k) == k:
                                minSup = top_k[-1][1]
                        # tiếp tục gọi hàm với cây vừa tạo ở trên để khai thác các mẫu tiếp theo
                        if self.calculate_twu(conditional_pattern_base, new_prefix) >= minUtil:
                            if conditional_pattern_tree.root.children:
                                self.mine_top_k_patterns(conditional_pattern_tree, k, minSup, minUtil, expected_support, new_prefix, top_k)
    
        return top_k



    def collect_paths(self, uhufp_tree, prefix: List[str]) -> List[Tuple[List[Tuple[str, float, float, float]], int]]:
        """Phương thức thu thập các đường dẫn của prefix

        Args:
            uhufp_tree (UHUFP_tree): cây UHUFP của prefix
            prefix (List[str]): mẫu cần thu thập đường dẫn

        Returns:
            List[Tuple[List[Tuple[str, float, float, float]], int]]: các đường dẫn và các thông tin của đường dẫn
        """
        paths = []
        
        # Lấy item cuối cùng của prefix
        last_item = prefix[-1] if prefix else None
        # print(f"LAST ITEM: {last_item}")
        if last_item and last_item in uhufp_tree.header_table:
            for node in uhufp_tree.header_table[last_item]:
                path = []
                current_node = node
                
                # duyệt từ node hiện tại về gốc của cây
                while current_node and current_node.item is not None:
                    path.append((current_node.item, current_node.prob, current_node.utility, current_node.nu))
                    if current_node.parent is None:
                        break
                    current_node = current_node.parent
                
                # Đảo ngược path để có thứ tự từ gốc đến node hiện tại
                path.reverse()
                
                # Kiểm tra xem path có chứa tất cả các item trong prefix không
                if all(item in [p[0] for p in path] for item in prefix):
                    paths.append((path, node.count))
        # for path, count in paths:
            # print(f"Path: {path}, Count: {count}")
        return paths



    def build_conditional_pattern_base(self, uhufp_tree, item: str, min_util: float) -> List[Tuple[Dict[str, float], Dict[str, float], int]]:
        """phương thức thu thập đường dần của item và tổ chức lại đường dẫn với chiến lược 7

        Args:
            uhufp_tree (UHUFP_tree): cây UHUFP của item
            item (str): mẫu cần thu thập đường dẫn
            min_util (float): ngưỡng tiện ích

        Returns:
            List[Tuple[Dict[str, float], Dict[str, float], int]]: _description_
        """
        paths = self.collect_paths(uhufp_tree, item)
        conditional_pattern_base = []

        # print(f"Paths for item {item}: {paths}")
        path_utilities_item = defaultdict(int)
        for path, count in paths:
            path_utility = path[-1][3]  # Total utility of the path

            # tính tiện ích đường dẫn của item trong đường dẫn
            for i in path:
                path_utilities_item[i[0]] += path_utility

            # print(f"Path: {path}")
            # print(f"Path utility: {path_utility}")
            # print(f"Path utilities item: {dict(path_utilities_item)}")
        path_util_item = 0
        # tính tiện ích đường dẫn của item trong CPB
        for i in item:
            path_util_item += path_utilities_item[i]
        unpromissing_item = []
        miu = {}
        # các giá trị tiện ích nhỏ nhất của các item trong path
        for path, count in paths:
            for i, s, u, _ in path:
                if i in miu:
                    miu[i] = min(miu[i], u)
                else:
                    miu[i] = u

        for path, count in paths:
            filtered_pu = 0
            for i, s, _, u in path:
                # áp dụng chiến lược 7 (DLU)
                if path_utilities_item[i] <= min_util:
                    filtered_pu += miu[i] * count
                    # print(f"Item: {i}, Node Utility: {u}, Min Utility: {miu[i]}, Count: {count}")
                    # print(f"Filtered Utility for item {i}: {filtered_pu}")
                    unpromissing_item.append((i,s,u))  
            
            reorganized_path = []

            for i, s, u, _ in path:
                if (i,s,u) not in unpromissing_item:
                    reorganized_path.append((i, s, u, path[-1][3] - filtered_pu))
            # sắp xếp danh sách path theo giá trị tiện ích đường dẫn của item trong CPB
            sorted_reorganized_path = sorted(reorganized_path, key=lambda x: path_utilities_item[x[0]], reverse=False)
            # print(f"Reorganized Path: {reorganized_path}")
            # print(f"Sorted Reorganized Path: {sorted_reorganized_path}")
        
            if sorted_reorganized_path:
                conditional_pattern_base.append((sorted_reorganized_path, count))
                # print(f"Conditional Pattern Base Entry: {conditional_pattern_base}")
        
        return conditional_pattern_base, miu, path_util_item



    def build_conditional_tree(self, conditional_pattern_base: List[Tuple[Tuple[str,float,int], int]], miu) -> 'UHUFP_tree':
        """Xây dựng cây UHUFP cho item từ CPB của item

        Args:
            conditional_pattern_base (List[Tuple[Tuple[str,float,int], int]]): chứa các đường dẫn của item
            miu (Dict[str:int]): danh sách các giá trị tiện ích nhỏ nhất của các item trong đường dẫn

        Returns:
            UHUFP_tree: cây UHUFP
        """
        
        conditional_tree = UHUFP_tree()

        for path, count in conditional_pattern_base:
            filtered_path = {item: support for item, support,_, _ in path}
            filtered_utilities = {item: util for item, _, util, _ in path}
            filtered_path_utility = {item: pu for item, _, _, pu in path}
            if filtered_path:  # nếu tồn tại path
                # print(f"Inserting transaction into conditional tree: Path: {filtered_path}, Utilities: {filtered_utilities}")
                conditional_tree.insert_reorganized_path(filtered_path, filtered_utilities, filtered_path_utility, miu)

        return conditional_tree


    def load_transactions_and_utilities(self, transaction_file: str, utility_file: str):
        """Đọc các cơ sở dữ liệu 

        Args:
            transaction_file (str): _description_
            utility_file (str): _description_

        Returns:
            Tuple[List[Dict[str, float]], List[Dict[str, float]], List[str], Dict[str, int], int]: các giá trị giao dịch chứa xác xuất và giá trị tiện ích
        """
        
        transactions = []
        utilities = []
        items = []
        dbUtil = 0
        item_twu = defaultdict(int)

        with open(transaction_file, mode='r') as tfile, open(utility_file, mode='r') as ufile:
            tlines = tfile.readlines()
            ulines = ufile.readlines()

            # Đọc file chứ dữ liệu xác xuất
            header = tlines[0].strip().split()
            items = [item.strip() for item in header]

            for line1, line2 in zip(tlines[1:], ulines):
                row = line1.strip().split()
                transaction = {items[i]: float(row[i]) for i in range(len(items)) if float(row[i]) > 0}
                transactions.append(transaction)

                parts = line2.strip().split(':')
                item_list = parts[0].split()
                utility_values = list(map(int, parts[2].split()))
                utility = {item_list[i]: utility_values[i] for i in range(len(item_list)) if item_list[i] in transaction}
                utilities.append(utility)
                dbUtil += int(parts[1])
                # print(transaction_utility)
                self.database_size += 1

            for utility in utilities:
                for item, util in utility.items():
                    item_twu[item] += sum(utility.values())
        
        return transactions, utilities, item, item_twu, dbUtil

    def build_tree_and_mine_patterns(self, transaction_file: str, utility_file: str, k: int, percentage: float, minSup: float):
        """Chạy thuật toán TUHUFP_Growth

        Args:
            transaction_file (str): các giao chứa xác xuất
            utility_file (str): giao chứa tiện ích
            k (int): số lượng mẫu cần khai thác
            percentage (float): ngưỡng tiện ích (%)
            minSup (float): ngưỡng phổ biến
        """
        
        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0
        transactions, utilities, items, item_twu, dbUtil = self.load_transactions_and_utilities(transaction_file, utility_file)
        


        promising_items = set()
        unpromising_items = set()
        reorganized_transaction = defaultdict(float)
        reorganized_utilities = defaultdict(int)

        
        uhufp_tree = UHUFP_tree()
        minUtil = int(dbUtil*percentage)
        # lấy danh sách các item hứa hẹn
        for item, twu in item_twu.items():
            if twu >= self.minUtil:
                promising_items.add(item)
            else:
                unpromising_items.add(item)

        # print(promising_items)

        for transaction, utility in zip(transactions, utilities):
            # print(transaction)
            # chiến lược 1
            reorganized_transaction = {item: prob for item, prob in transaction.items() if item in promising_items}
            reorganized_utilities = {item: util for item, util in utility.items() if item in reorganized_transaction}
            sorted_items = sorted(reorganized_transaction.keys(), key=lambda item: item_twu[item], reverse=True)
            # print(sorted_items)
            # xây dựng cây UHUFP
            uhufp_tree.insert_reorganized_transaction(reorganized_transaction, reorganized_utilities, sorted_items)

        # uhufp_tree.print_header_table()
        # uhufp_tree.prune_patterns(k, item_twu, minUtil)
        # uhufp_tree.prune_tree(uhufp_tree.root)
        
        # print({'database util: ': dbUtil})
        print({'min util: ': minUtil})

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        self.top_k_UHUFP = self.mine_top_k_patterns(uhufp_tree, k, minSup, minUtil, 1.0)
        self.minUtil = minUtil
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()
        # print("\nTop-K High Utility Frequent Patterns:")
        for i, (pattern, expSup, utility) in enumerate(self.top_k_UHUFP, 1):
            print(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}")

    def print_stats(self, path):
        """Thống kê thông tin kết quả chạy thuật toán

        Args:
            path (str): đường dẫn lưu thông tin
        """
        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.minUtil}\n")
            for i, (pattern, expSup, utility) in enumerate(self.top_k_UHUFP, 1):
                output_file.write(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage
# Example usage

In [6]:
# chạy với ví dụ mẫu
transaction_file = '../../data/example.txt'
utility_file = '../../data/example_utility.txt'
k = 6
percentage = 0.3
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_example.txt")

{'min util: ': 30}
1. Pattern: ('d',), ExpSup: 3.3, Utility: 30
2. Pattern: ('c', 'd'), ExpSup: 1.95, Utility: 38
3. Pattern: ('a', 'd'), ExpSup: 1.2, Utility: 35
4. Pattern: ('a', 'c', 'd'), ExpSup: 0.98, Utility: 42
5. Pattern: ('b', 'c', 'd'), ExpSup: 0.88, Utility: 40
6. Pattern: ('a', 'b', 'd'), ExpSup: 0.55, Utility: 33


In [7]:
# chạy với foodmart
transaction_file = '../../data/input_foodmart.txt'
utility_file = '../../data/foodmart_utility.txt'
k = 300
percentage = 0.0004
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_foodmart_top_900.txt")

{'min util: ': 4804}
1. Pattern: ('545',), ExpSup: 12.1, Utility: 9419
2. Pattern: ('634',), ExpSup: 11.88, Utility: 11881
3. Pattern: ('1423',), ExpSup: 11.75, Utility: 16116
4. Pattern: ('617',), ExpSup: 11.75, Utility: 7527
5. Pattern: ('916',), ExpSup: 11.68, Utility: 15163
6. Pattern: ('903',), ExpSup: 11.62, Utility: 14616
7. Pattern: ('888',), ExpSup: 11.38, Utility: 5280
8. Pattern: ('1390',), ExpSup: 11.26, Utility: 12415
9. Pattern: ('404',), ExpSup: 11.02, Utility: 7000
10. Pattern: ('308',), ExpSup: 10.94, Utility: 11554
11. Pattern: ('1351',), ExpSup: 10.9, Utility: 9352
12. Pattern: ('1399',), ExpSup: 10.11, Utility: 10324
13. Pattern: ('378',), ExpSup: 10.08, Utility: 5994
14. Pattern: ('69',), ExpSup: 10.07, Utility: 16430
15. Pattern: ('749',), ExpSup: 10.03, Utility: 11748
16. Pattern: ('1426',), ExpSup: 9.96, Utility: 21692
17. Pattern: ('200',), ExpSup: 9.8, Utility: 12190
18. Pattern: ('326',), ExpSup: 9.77, Utility: 13091
19. Pattern: ('1347',), ExpSup: 9.71, Util

In [8]:
# chạy với retail
transaction_file = '../../data/input_retail.txt'
utility_file = '../../data/retail_utility.txt'
k = 300
percentage = 0.0001
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_retail_top_300.txt")

KeyboardInterrupt: 

In [ ]:
# chạy với chess
transaction_file = '../../data/input_chess.txt'
utility_file = '../../data/chess_utility.txt'
k = 300
percentage = 0.001
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_chess_top_300.txt")